In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('./fire_archive.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(8, 8), dpi=120)
sns.heatmap(df.corr(), annot=True, cmap='gist_rainbow')
plt.show()

In [ ]:
df.drop(['track'], axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
# finding categorical data
df['scan'].value_counts()

In [ ]:
df['acq_time'].value_counts()

In [ ]:
df['satellite'].value_counts()

In [ ]:
df['instrument'].value_counts()

In [ ]:
df['version'].value_counts()

In [ ]:
df['daynight'].value_counts()

In [ ]:
df.drop(['instrument', 'version'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df['daynight'] = df['daynight'].map({'D': 1, 'N':0})
df['satellite'] = df['satellite'].map({"Terra": 1, "Aqua": 0})

In [ ]:
df.head()

In [ ]:
df = pd.get_dummies(data=df, columns=['type'], drop_first=True)

In [ ]:
df.head()

In [ ]:
# binning the scan columns

bins= [0, 1, 2, 3, 4, 5]
labels = [1, 2, 3, 4, 5]

df['scan_bin'] = pd.cut(df['scan'], bins=bins, labels=labels)

In [ ]:
df.head()

In [ ]:
df['acq_date'] = pd.DatetimeIndex(df['acq_date'])
df['month'] = df['acq_date'].apply(lambda x: x.month)

In [ ]:
df.head()

In [ ]:
df.drop(['scan', 'acq_date', 'acq_time', 'bright_t31'], axis=True, inplace=True)

In [ ]:
plt.figure(figsize=(15, 8), dpi=90)
sns.heatmap(df.corr(), annot=True, cmap='gist_rainbow')
plt.show()

In [ ]:
df.head()

In [ ]:
x = df.drop('confidence', axis=1).values
y = df['confidence'].values

In [ ]:
scaler = StandardScaler().fit(x)
x = scaler.transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=101)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
models = [LinearRegression, RandomForestRegressor]

In [ ]:
def get_model_score(x):
  model = x()
  model.fit(x_train, y_train)
  print(type(model).__name__)
  print('the training score is', model.score(x_train, y_train))
  y_pred = model.predict(x_test)
  print('the Testing score is', metrics.r2_score(y_test, y_pred))
  return model

In [ ]:
all_models = {}
for i in models:
  model = get_model_score(i)
  all_models[type(model).__name__] = model

In [ ]:
# hyperparameter tuning for random forest regression
model = all_models['RandomForestRegressor']

In [ ]:
model.get_params()

In [ ]:
n_estimators = [int(x) for x in np.linspace(start=300, stop=1000, num=35)]
max_features = ['auto', 'sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(15, 35, num=7)]
max_depth.append(None)
min_samples_split= [2, 3, 5, 6]
min_samples_leaf = [1, 3, 5, 7]


In [ ]:
random_grid = {
    'n_estimators':n_estimators,
    'max_features':max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf' : min_samples_leaf
}


In [ ]:
rf = RandomizedSearchCV(model, random_grid, n_iter=50, cv=3, verbose=1, random_state=42)

In [ ]:
rf.fit(x_test, y_test)

In [ ]:
rf.best_params_

In [ ]:
model = RandomForestRegressor(n_estimators=315, min_samples_split=6, min_samples_leaf = 1,
 max_features='sqrt', max_depth = None)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
model.score(x_train, y_train)

In [ ]:
model.score(x_test, y_test)